## Prerequisites - Environment Setup

Before starting this workshop, you need to set up a .NET development environment. Follow these steps:

### Step 1: Install .NET 9 SDK
Download and install the latest .NET 9 SDK from [https://dotnet.microsoft.com/download/dotnet/9.0](https://dotnet.microsoft.com/download/dotnet/9.0)

### Step 2: Install .NET Interactive Extension
In VS Code, install the ".NET Interactive Notebooks" extension to run C# code in Jupyter notebooks.

### Step 3: Create .env Configuration File
Create a `.env` file in this directory (or the parent directory) with your Azure OpenAI credentials:
```bash
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini
```

**Important:** Never commit your `.env` file to version control! Add it to `.gitignore`.

### Step 4: Select .NET Interactive Kernel
1. Click on the **kernel selector** in the top-right corner of this notebook
2. Select **".NET Interactive"**
3. Choose **"C#"** as the language

---

# .NET Semantic Kernel Agents Workshop

This notebook demonstrates modern AI agent development using Microsoft Semantic Kernel in .NET 9 with Azure OpenAI services.

## Workshop Focus Areas
1. **ChatCompletionAgent** - Build generic agents using the latest Semantic Kernel agent framework
2. **Azure AI Foundry Agents** - Demonstrate specialized agents like PeopleLookupAgent
3. **Group Chat System** - Multi-agent collaboration with built-in orchestration

## Key Technologies
- **Semantic Kernel 1.65.0** - Microsoft's latest AI orchestration framework
- **ChatCompletionAgent** - Modern agent pattern for conversational AI
- **Azure OpenAI** - GPT-4 models for intelligent responses
- **Agent Group Chat** - Built-in multi-agent coordination

Let's build sophisticated AI agent systems using the latest Semantic Kernel patterns!

### Step 1: Verify .NET Environment

In [ ]:
// Step 1: Verify .NET Environment
// Check if .NET Interactive is working properly

using System;
using System.Runtime.InteropServices;

Console.WriteLine("=== .NET Environment Check ===");
Console.WriteLine($".NET Version: {Environment.Version}");
Console.WriteLine($"Runtime: {RuntimeInformation.FrameworkDescription}");
Console.WriteLine($"OS: {RuntimeInformation.OSDescription}");
Console.WriteLine($"Architecture: {RuntimeInformation.OSArchitecture}");
Console.WriteLine();
Console.WriteLine("✓ .NET Interactive is working!");
Console.WriteLine("Ready to install Semantic Kernel packages!");

// Return a value to ensure cell execution is visible
Console.WriteLine("Environment check completed successfully!");
"SUCCESS: .NET Interactive is running properly"

### Step 2: Install Semantic Kernel Packages

In [ ]:
// Step 2: Install Required NuGet Packages
// Install the latest Semantic Kernel packages for agent development

Console.WriteLine("Installing Semantic Kernel packages...");

// Core Semantic Kernel package
#r "nuget: Microsoft.SemanticKernel, 1.65.0"

// Agents package for multi-agent scenarios  
#r "nuget: Microsoft.SemanticKernel.Agents.OpenAI, 1.65.0"

// Azure OpenAI connector
#r "nuget: Microsoft.SemanticKernel.Connectors.OpenAI, 1.65.0"

// .NET Configuration and Environment support
#r "nuget: DotNetEnv, 3.1.1"

// Additional useful packages
#r "nuget: Microsoft.Extensions.Logging, 9.0.0"
#r "nuget: Microsoft.Extensions.Logging.Console, 9.0.0"
#r "nuget: Microsoft.Extensions.DependencyInjection, 9.0.0"

Console.WriteLine("✓ All Semantic Kernel packages installed successfully!");
Console.WriteLine("Ready to import namespaces and create agents!")

### Troubleshooting: If You Don't See Output

**If the Run button does nothing or no output appears:**

#### 🚨 Most Common Issue: Kernel Not Selected
1. **Look at the top-right corner** of the notebook - you should see something like ".NET Interactive" or "C#"
2. **If you see "Select Kernel"** or a different kernel:
   - Click on it
   - Select **".NET Interactive"**
   - Choose **"C#"** from the sub-menu

#### 🔧 Other Solutions to Try:

1. **Install .NET Interactive Extension**:
   - Press `Ctrl+Shift+X` to open Extensions
   - Search for **".NET Interactive Notebooks"** by Microsoft
   - Install if not already installed
   - **Reload VS Code** after installation

2. **Restart Everything**:
   - Press `Ctrl+Shift+P`
   - Type "Developer: Reload Window"
   - Reopen the notebook and try again

3. **Check .NET Installation**:
   - Open a terminal (`Ctrl+Shift+``)
   - Run: `dotnet --version`
   - Should show .NET 8.0 or later

4. **Force Kernel Selection**:
   - Press `Ctrl+Shift+P`
   - Type "Notebook: Select Notebook Kernel"
   - Choose ".NET Interactive" → "C#"

**The cell below should show "42" as output if everything is working.**

### Step 3: Automatic .env Configuration

This notebook automatically loads your Azure OpenAI configuration from a `.env` file, making it much easier for beginners! 

**How it works:**
1. The notebook searches for a `.env` file in the current directory or parent directories
2. It automatically loads all environment variables from the file
3. No need to manually set environment variables or edit code

**Sample .env file:**
```
AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/
AZURE_OPENAI_API_KEY=your_api_key_here
AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini
```

**Benefits of .env approach:**
- ✅ No code modification needed
- ✅ Secure credential management
- ✅ Easy to share configurations
- ✅ Matches Python workshop pattern
- ✅ Beginner-friendly setup

**Next:** Import Required Namespaces

In [ ]:
// Step 3: Import Required Namespaces
// Import all the Semantic Kernel namespaces for modern agent development

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Agents;
using Microsoft.SemanticKernel.Agents.OpenAI;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using Microsoft.Extensions.Logging;
using Microsoft.Extensions.DependencyInjection;
using System.ComponentModel;
using System.IO;
using DotNetEnv;

Console.WriteLine("✓ All namespaces imported successfully!");
Console.WriteLine("Ready to configure Azure OpenAI and create modern agents!");

### Step 4: Configure Azure OpenAI

In [ ]:
// Step 4: Configure Azure OpenAI Connection
// Automatically load configuration from .env file

Console.WriteLine("Configuring Azure OpenAI...");

// Load .env file from current directory or parent directories
try
{
    // Try loading .env from current notebook directory
    var notebookDir = Directory.GetCurrentDirectory();
    var envFile = Path.Combine(notebookDir, ".env");
    
    if (File.Exists(envFile))
    {
        Env.Load(envFile);
        Console.WriteLine($"✓ Loaded .env file from: {envFile}");
    }
    else
    {
        // Try parent directories (common pattern)
        var parentDir = Directory.GetParent(notebookDir);
        while (parentDir != null)
        {
            envFile = Path.Combine(parentDir.FullName, ".env");
            if (File.Exists(envFile))
            {
                Env.Load(envFile);
                Console.WriteLine($"✓ Loaded .env file from: {envFile}");
                break;
            }
            parentDir = parentDir.Parent;
        }
        
        if (parentDir == null)
        {
            Console.WriteLine("! No .env file found, using system environment variables");
        }
    }
}
catch (Exception ex)
{
    Console.WriteLine($"! Error loading .env file: {ex.Message}");
    Console.WriteLine("Falling back to system environment variables");
}

// Load configuration from environment variables (now populated from .env)
var azureOpenAIEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
var azureOpenAIApiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
var deploymentName = Environment.GetEnvironmentVariable("AZURE_OPENAI_DEPLOYMENT_NAME") ?? "gpt-4o-mini";

// Required environment variables
var requiredVars = new Dictionary<string, string?>
{
    { "AZURE_OPENAI_ENDPOINT", azureOpenAIEndpoint },
    { "AZURE_OPENAI_API_KEY", azureOpenAIApiKey },
    { "AZURE_OPENAI_DEPLOYMENT_NAME", deploymentName }
};

// Check if all required variables are present
var missingVars = requiredVars.Where(kvp => string.IsNullOrWhiteSpace(kvp.Value)).Select(kvp => kvp.Key).ToList();

if (missingVars.Any())
{
    Console.WriteLine("❌ Missing required Azure OpenAI configuration!");
    Console.WriteLine($"Missing variables: {string.Join(", ", missingVars)}");
    Console.WriteLine();
    Console.WriteLine("To fix this, create a .env file in this directory (or parent directory) with:");
    Console.WriteLine("AZURE_OPENAI_ENDPOINT=https://your-resource.openai.azure.com/");
    Console.WriteLine("AZURE_OPENAI_API_KEY=your_api_key_here");
    Console.WriteLine("AZURE_OPENAI_DEPLOYMENT_NAME=gpt-4o-mini");
    Console.WriteLine();
    Console.WriteLine("Then restart this notebook kernel and run this cell again.");
    throw new InvalidOperationException("Required Azure OpenAI configuration missing!");
}

Console.WriteLine("✓ Azure OpenAI configuration loaded successfully!");
Console.WriteLine($"Endpoint: {azureOpenAIEndpoint}");
Console.WriteLine($"Deployment: {deploymentName}");
Console.WriteLine("Ready to create Semantic Kernel!");

### Step 5: Create Semantic Kernel Instance

In [ ]:
// Step 5: Create Semantic Kernel Instance
// Set up the Semantic Kernel with Azure OpenAI chat completion

Console.WriteLine("Creating Semantic Kernel...");

// Create Kernel Builder
var builder = Kernel.CreateBuilder();

// Add Azure OpenAI Chat Completion service
builder.AddAzureOpenAIChatCompletion(
    deploymentName: deploymentName,
    endpoint: azureOpenAIEndpoint,
    apiKey: azureOpenAIApiKey);

// Add logging
builder.Services.AddLogging(c => c.AddConsole().SetMinimumLevel(LogLevel.Information));

// Build the kernel
var kernel = builder.Build();

Console.WriteLine("✓ Semantic Kernel created successfully!");
Console.WriteLine("Ready to create our first agent!");

In [ ]:
// Step 6: Create Your First ChatCompletionAgent
// This demonstrates the modern Semantic Kernel agent pattern

Console.WriteLine("Creating ChatCompletionAgent...");

// Create a generic ChatCompletionAgent
var genericAgent = new ChatCompletionAgent()
{
    Name = "GenericAssistant",
    Instructions = @"You are a helpful AI assistant that can answer questions and have conversations.
    
    Your capabilities include:
    - Answering general questions
    - Helping with problem-solving
    - Providing explanations and information
    - Having natural conversations
    
    Always be helpful, accurate, and professional in your responses.",
    Kernel = kernel
};

Console.WriteLine("✓ ChatCompletionAgent created successfully!");
Console.WriteLine($"Agent Name: {genericAgent.Name}");
Console.WriteLine($"Agent Type: {genericAgent.GetType().Name}");
Console.WriteLine("Ready to test the modern agent pattern!");

### Test Your First Agent

In [ ]:
// Test the ChatCompletionAgent
// Send a message to the agent and see its response

Console.WriteLine("Testing the ChatCompletionAgent...");

// Create a chat history for the conversation
var chatHistory = new ChatHistory();

// Test message - you can modify this to ask different questions
var userMessage = "Hello! What is artificial intelligence and how does it work?";

Console.WriteLine($"User: {userMessage}");
chatHistory.AddUserMessage(userMessage);

try
{
    // Get agent response using the modern agent pattern
    await foreach (var response in genericAgent.InvokeAsync(chatHistory))
    {
        Console.WriteLine($"Agent ({genericAgent.Name}): {response.Content}");
        // Add the agent's response to chat history for context
        chatHistory.Add(response);
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ ChatCompletionAgent test completed successfully!");
    Console.WriteLine("You can modify the userMessage above to test different questions.");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error testing agent: {ex.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration.");
}

### Create Azure AI Foundry Agents

In [ ]:
// Create Azure AI Foundry Specialized Agents
// These agents demonstrate the integration with Azure AI Foundry services

Console.WriteLine("Creating Azure AI Foundry specialized agents...");

// People Lookup Agent - Expert at finding information about people
var peopleAgent = new ChatCompletionAgent()
{
    Name = "PeopleLookupAgent",
    Instructions = @"You are a People Lookup Agent, an expert at finding and providing information about people, contacts, and team members.

Your expertise includes:
• Directory Searches: Finding people by name, role, department, or skills
• Team Coordination: Understanding organizational structure and relationships  
• Contact Discovery: Providing appropriate contact information
• Role Identification: Matching people to specific tasks or expertise needs

Guidelines for responses:
- Be helpful and accurate when providing people information
- If you don't know a specific person, explain what information would be helpful to find them
- Suggest ways to locate people or get contact information
- Provide guidance on organizational structure and team dynamics
- Always try to be helpful for any people-related query

Response Style:
- Professional yet approachable
- Clear and organized information
- Include relevant context about why someone might be the right contact",
    Kernel = kernel
};

// Knowledge Finder Agent - Expert at searching and retrieving information
var knowledgeAgent = new ChatCompletionAgent()
{
    Name = "KnowledgeFinderAgent", 
    Instructions = @"You are a Knowledge Finder Agent, a specialist at searching, retrieving, and organizing information from various knowledge sources.

Your expertise includes:
• Document Search: Finding relevant documents, wikis, and knowledge bases
• Research Assistance: Discovering and synthesizing information from multiple sources
• Information Organization: Structuring and summarizing complex knowledge
• Context Provision: Connecting related information and concepts
• Relevance Filtering: Identifying the most pertinent information for specific needs

Guidelines for responses:
- Focus on finding the most relevant and accurate information
- Provide clear summaries with key points highlighted
- Help users understand complex information through clear explanations
- Suggest additional related topics or resources for deeper learning
- Structure information in a logical, easy-to-digest format

Response Style:
- Well-organized with clear headings and bullet points
- Include actionable insights and takeaways
- Provide both high-level summaries and detailed explanations as needed",
    Kernel = kernel
};

Console.WriteLine("✓ Azure AI Foundry agents created successfully!");
Console.WriteLine($"1. {peopleAgent.Name} - Specializes in people and contact information");
Console.WriteLine($"2. {knowledgeAgent.Name} - Expert at finding and organizing knowledge");
Console.WriteLine();
Console.WriteLine("These agents demonstrate specialized capabilities that could integrate with Azure AI Foundry services!");

In [ ]:
// Test Azure AI Foundry Agents
// Let's test each specialized agent with questions in their area of expertise

Console.WriteLine("Testing Azure AI Foundry specialized agents...");
Console.WriteLine();

// Test cases for each agent
var testCases = new[]
{
    new { Agent = peopleAgent, Question = "I need to find someone who can help with .NET development. Who should I contact?" },
    new { Agent = knowledgeAgent, Question = "What are the best practices for implementing microservices architecture?" }
};

for (int i = 0; i < testCases.Length; i++)
{
    var testCase = testCases[i];
    
    try
    {
        Console.WriteLine($"Test {i + 1}: {testCase.Agent.Name}");
        Console.WriteLine($"Question: {testCase.Question}");
        Console.WriteLine(new string('-', 50));
        
        // Create chat history for this test
        var testChatHistory = new ChatHistory();
        testChatHistory.AddUserMessage(testCase.Question);
        
        // Get response from the agent
        await foreach (var response in testCase.Agent.InvokeAsync(testChatHistory))
        {
            Console.WriteLine($"Response: {response.Content}");
        }
        
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
    catch (Exception ex)
    {
        Console.WriteLine($"❌ Error testing {testCase.Agent.Name}: {ex.Message}");
        Console.WriteLine(new string('=', 70));
        Console.WriteLine();
    }
}

Console.WriteLine("✓ Specialized agent testing completed!");
Console.WriteLine("Each agent demonstrated expertise in their specialized domain.");

### Create Group Chat with All Three Agents

In [ ]:
// Create Group Chat with Modern AgentGroupChat
// This demonstrates multi-agent collaboration using Semantic Kernel's built-in group chat

Console.WriteLine("Creating Agent Group Chat...");

// Create an AgentGroupChat with all three agents
var groupChat = new AgentGroupChat(genericAgent, peopleAgent, knowledgeAgent)
{
    // Configure the execution settings for the group chat
    ExecutionSettings = new OpenAIPromptExecutionSettings()
    {
        ToolCallBehavior = ToolCallBehavior.AutoInvokeKernelFunctions,
        Temperature = 0.7,
        MaxTokens = 1000
    }
};

Console.WriteLine("✓ Agent Group Chat created successfully!");
Console.WriteLine($"Participants: {string.Join(", ", new[] { genericAgent.Name, peopleAgent.Name, knowledgeAgent.Name })}");
Console.WriteLine("Ready for multi-agent collaboration!");

// Test the group chat with a complex question that benefits from multiple perspectives
var complexQuestion = @"We're launching a new AI-powered project management tool. 
We need to understand:
1. Who should be involved in the development team?
2. What technical knowledge and best practices should we consider?
3. How can we ensure the project is successful?

Please provide a comprehensive response with insights from different perspectives.";

Console.WriteLine();
Console.WriteLine("Testing Group Chat with Complex Question:");
Console.WriteLine($"Question: {complexQuestion}");
Console.WriteLine(new string('=', 80));

try
{
    // Add the user question to the group chat
    groupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, complexQuestion));
    
    Console.WriteLine("Starting group chat collaboration...");
    
    // Track the conversation
    int messageCount = 0;
    const int maxMessages = 6; // Limit to prevent too long conversations
    
    // Process the group chat conversation
    await foreach (var message in groupChat.InvokeAsync())
    {
        messageCount++;
        var author = message.AuthorName ?? "Unknown";
        
        Console.WriteLine($"\n[{messageCount}] {author}:");
        Console.WriteLine($"{message.Content}");
        Console.WriteLine(new string('-', 60));
        
        // Break if we've reached the message limit
        if (messageCount >= maxMessages)
        {
            Console.WriteLine("\n[Conversation limit reached for demo purposes]");
            break;
        }
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ Group Chat completed successfully!");
    Console.WriteLine($"Generated {messageCount} messages from multiple agents working together!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in group chat: {ex.Message}");
    Console.WriteLine("Please check your Azure OpenAI configuration and try again.");
}

### Test Group Chat System

In [ ]:
// Try Your Own Group Chat Question
// Modify the question below to explore different multi-agent scenarios

Console.WriteLine("Custom Group Chat Test");
Console.WriteLine("Feel free to modify the question below to explore different topics!");
Console.WriteLine();

// MODIFY THIS QUESTION to test different scenarios:
var customQuestion = @"How can we improve remote team collaboration? 
Consider finding the right people, knowledge management, and general best practices.";

Console.WriteLine($"Your Question: {customQuestion}");
Console.WriteLine(new string('=', 60));

try
{
    // Create a new group chat for this custom question
    var customGroupChat = new AgentGroupChat(genericAgent, peopleAgent, knowledgeAgent);
    
    // Add the custom question
    customGroupChat.AddChatMessage(new ChatMessageContent(AuthorRole.User, customQuestion));
    
    Console.WriteLine("Starting custom group chat...");
    
    // Track the conversation
    int messageCount = 0;
    const int maxMessages = 4; // Shorter for custom test
    
    // Process the group chat conversation
    await foreach (var message in customGroupChat.InvokeAsync())
    {
        messageCount++;
        var author = message.AuthorName ?? "Unknown";
        
        Console.WriteLine($"\n[{messageCount}] {author}:");
        Console.WriteLine($"{message.Content}");
        Console.WriteLine(new string('-', 40));
        
        // Break if we've reached the message limit
        if (messageCount >= maxMessages)
        {
            Console.WriteLine("\n[Demo limit reached]");
            break;
        }
    }
    
    Console.WriteLine();
    Console.WriteLine("✓ Custom group chat completed!");
    Console.WriteLine("Feel free to modify the customQuestion above to explore different topics!");
}
catch (Exception ex)
{
    Console.WriteLine($"❌ Error in custom group chat: {ex.Message}");
}

## Workshop Summary

Congratulations! You've successfully built modern AI agents using the latest Semantic Kernel patterns:

### What You've Learned:

1. **ChatCompletionAgent** - The modern way to create conversational agents in Semantic Kernel
   - Simple, clean API for agent creation
   - Built-in conversation management
   - Easy integration with Azure OpenAI

2. **Azure AI Foundry Agents** - Specialized agents with expert knowledge
   - PeopleLookupAgent for finding contacts and team information
   - KnowledgeFinderAgent for research and information retrieval
   - Focused expertise with clear role definitions

3. **AgentGroupChat** - Multi-agent collaboration made simple
   - Built-in orchestration and coordination
   - Natural conversation flow between agents
   - Automatic turn-taking and response synthesis

### Key Benefits of This Approach:
- ✅ **Modern Pattern** - Uses latest Semantic Kernel 1.30.0 features
- ✅ **Production Ready** - Same patterns used in the backend API
- ✅ **Simplified Code** - Less boilerplate, more functionality
- ✅ **Azure Integration** - Seamless Azure OpenAI connectivity

### Next Steps:
- Explore the backend API at `/Backend/dotnet/sk/`
- Try different agent instructions and personalities
- Experiment with more complex group chat scenarios
- Build your own specialized agents for specific domains

You now have the foundation to build sophisticated multi-agent AI systems!